ACCL Initialization (Emulator)

In [ ]:
from pyaccl import accl, ACCLBuffer

START_PORT = 5500
WORLD_SIZE = 4
RXBUF_SIZE = 16*1024
PROTOCOL = "TCP"
COUNT = 1000

ranks = []
for i in range(WORLD_SIZE):
    ranks.append({"ip": "127.0.0.1", "port": START_PORT+WORLD_SIZE+i, "session_id":i, "max_segment_size": RXBUF_SIZE})

accl_instances = []
for i in range(WORLD_SIZE):
    accl_instances[i] = accl(   ranks, i, 
                                bufsize=RXBUF_SIZE,
                                protocol=PROTOCOL,
                                sim_sock="tcp://localhost:"+str(START_PORT+i)   )
op0_buffers = []
op0_buffers = []
res_buffers = []
for i in range(WORLD_SIZE):    
    op0_buffers[i] = ACCLBuffer((COUNT,), target=accl_instances[i].cclo.devicemem)
    res_buffers[i] = ACCLBuffer((COUNT,), target=accl_instances[i].cclo.devicemem)
    op0_buffers[i][:] = [1.0*i for i in range(COUNT)]


In [ ]:
import threading
from pyaccl import ACCLReduceFunctions
import numpy as np

def allreduce(n):
    accl_instances[n].allreduce(op0_buffers[n], res_buffers[n], COUNT, ACCLReduceFunctions.SUM)

threads = []
for i in range(WORLD_SIZE):
    threads[i] = threading.Thread(target=allreduce, args=(i,))
    threads[i].start()

for i in range(WORLD_SIZE):
    threads[i].join()
    assert np.isclose(res_buffers[i], sum(op0_buffers))

In [ ]:
for i in range(WORLD_SIZE):
    accl_instances[i].deinit()